In [1]:
import sys
import os

# Go up to the `app/` folder
sys.path.append(os.path.abspath(".."))

from db.database import Database

In [ ]:
ls

In [ ]:
ls ..

In [ ]:
ls ../db

In [2]:
def get_mode(row):
    if row['enhanced_prompt'] and row['used_suggestion']:
        return 'both'
    elif row['enhanced_prompt']:
        return 'enhancement'
    elif row['used_suggestion']:
        return 'suggestions'
    return 'noai'

def filter_by_tab(df, tab):
    if tab == 'overall':
        return df
    df['mode'] = df.apply(get_mode, axis=1)
    return df[df['mode'] == tab]

In [ ]:
with Database() as db:
        df_images = db.fetch_all_images()
        df_prompts = db.fetch_all_prompts()
df = df_images.merge(df_prompts, left_on="output_prompt_id", right_on="id", suffixes=("_img", ""))
df = filter_by_tab(df, tab)

In [ ]:
dataset_path = os.path.abspath(os.path.join("..", "db", "mma.db"))

In [ ]:
def get_mode(row):
    if row['enhanced_prompt'] and row['used_suggestion']:
        return 'Both'
    elif row['enhanced_prompt']:
        return 'Enhancement Only'
    elif row['used_suggestion']:
        return 'Suggestion Only'
    return 'Baseline'

def filter_by_mode(df, mode):
    if mode == 'overview':
        return df

    df['interaction_mode'] = df.apply(get_mode, axis=1)
    return df[df['interaction_mode'] == mode]

In [ ]:
with Database(dataset_path) as db:
        df_images = db.fetch_all_images()
        df_prompts = db.fetch_all_prompts()
        df = df_images.merge(df_prompts, left_on="prompt_id", right_on="id", suffixes=("_image", "_prompt"))
        print(df)

In [ ]:
mode = "Both"
df_filtered = filter_by_mode(df, mode)

In [ ]:
print(df_filtered)

In [ ]:
with Database(dataset_path) as db:  # adjust path if needed
    print("=== USERS ===")
    print(db.fetch_all_users())

    print("\n=== CHATS ===")
    print(db.fetch_all_chats())

    print("\n=== PROMPTS ===")
    print(db.fetch_all_prompts())

    print("\n=== IMAGES ===")
    print(db.fetch_all_images())


In [ ]:
with Database(dataset_path) as db:
    df_images = db.fetch_all_images()
    df_prompts = db.fetch_all_prompts()

    df = df_images.merge(
        df_prompts,
        left_on="prompt_id",
        right_on="id",
        suffixes=("_image", "_prompt")
    )

    # Drop redundant columns where values are always equal
    for col in ["user_id", "chat_id", "id"]:
        col_image = f"{col}_image"
        col_prompt = f"{col}_prompt"
        if col_image in df.columns and col_prompt in df.columns:
            if df[col_image].equals(df[col_prompt]):
                df.drop(columns=[col_prompt], inplace=True)
                df.rename(columns={col_image: col}, inplace=True)

    # Rename merged `id` from prompt table if kept
    if 'id' not in df.columns and 'id_image' in df.columns:
        df.rename(columns={'id_image': 'id'}, inplace=True)

print(df)

In [ ]:
with Database(dataset_path) as db:
    df_images = db.fetch_all_images()
    df_prompts = db.fetch_all_prompts()

# Merge on prompt_id
df = df_images.merge(
    df_prompts,
    left_on="prompt_id",
    right_on="id",
    suffixes=("_image", "_prompt")
)

# Rename prompt ID to keep clarity
df.rename(columns={"id_image": "image_id", "id_prompt": "prompt_id"}, inplace=True)

# Drop duplicate columns if values match
for col in ['user_id', 'chat_id']:
    col_img = f"{col}_image"
    col_prm = f"{col}_prompt"
    if col_img in df.columns and col_prm in df.columns:
        if df[col_img].equals(df[col_prm]):
            df.drop(columns=[col_prm], inplace=True)
            df.rename(columns={col_img: col}, inplace=True)

# Now you have a clean DataFrame
print(df.columns)
print(df[:7])